# Create response with michis old code (mic-tools)

In [8]:
from subprocess import check_output
from os.path import expanduser
import os

home = expanduser("~")
DATA_DIR = os.path.join(home, 'Devel/test/samuel/in4michi')
OUT_DIR = os.path.join(DATA_DIR, '../mictools_results')

SCRIPT = 'python2 /home/olivier/Devel/mic-tools/scripts/msmt-deconv/shore-response.py '
venv = '. ' + os.path.join(home, 'Devel/venvs/bonndit/bin/activate') + ' && '

command = venv + SCRIPT + '{dire}/ {out}/'.format(dire=DATA_DIR, out = OUT_DIR)
print(command)
out = check_output(command, shell = True)

for line in out.splitlines():
    print(line.decode("utf-8") )

. /home/olivier/Devel/venvs/bonndit/bin/activate && python2 /home/olivier/Devel/mic-tools/scripts/msmt-deconv/shore-response.py /home/olivier/Devel/test/samuel/in4michi/ /home/olivier/Devel/test/samuel/in4michi/../mictools_results/
('shore path:', '/home/olivier/Devel/mic-tools/packages/dwmri/shore.pyc')
4 4
/home/olivier/Devel/test/samuel/in4michi/dti_FA.nii.gz
Using provided DTI mask.
loading data....
reshaping...
csf response:
('  0.0%',)
 (' 43.6%',)
 (' 59.4%',)
    done   
13169 voxel
[  6.63152513e+04  -3.96555644e+04   9.53249881e+04  -3.67208476e+02
  -7.35905419e+01   1.05103369e+02  -2.53682288e+01   5.80324180e+01
   1.06293846e+03   4.50459828e+01   6.66277144e+02   3.63562321e+02
  -1.20116268e+02   7.02977228e+01   3.25583620e+01   4.52711879e+00
  -3.07358453e+01  -1.86486783e+01  -1.10549390e+01  -3.39255786e+00
  -1.91630816e+00   3.45451323e+01]
[ 66315.25134601 -39655.56435794  95324.98808202]
grey matter response:
('  0.0%',)
 (' 29.4%',)
 (' 38.3%',)
 (' 48.7%',)


# Create response with bonndit

In [3]:
from subprocess import check_output
from os.path import expanduser
import os

home = expanduser("~")
DATA_DIR = os.path.join(home, 'Devel/test/samuel/in4michi/')
OUT_DIR = os.path.join(DATA_DIR, '../bonndit_results')

SCRIPT = 'mtdeconv '
venv = '. ' + os.path.join(home, 'Devel/venvs/bonndit/bin/activate') + ' && '

command = venv + SCRIPT + '{dire}/ -o {out}/ -Rv'.format(dire=DATA_DIR, out = OUT_DIR)
print(command)
out = check_output(command, shell = True)

for line in out.splitlines():
    print(line.decode("utf-8") )

. /home/olivier/Devel/venvs/bonndit/bin/activate && mtdeconv /home/olivier/Devel/test/samuel/in4michi// -o /home/olivier/Devel/test/samuel/in4michi/../bonndit_results/ -Rv


In [4]:
import numpy as np
import importlib.util
import nibabel as nib
from dipy.core.gradients import gradient_table
from dipy.io import read_bvals_bvecs
import bonndit as bd
import os

In [5]:
spec = importlib.util.spec_from_file_location("shore", "/home/olivier/Devel/mic-tools/packages/dwmri/shore.py")
mic_shore = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mic_shore)

In [63]:
DATA_DIR = "/home/olivier/Devel/test/samuel/in4michi/"

zeta = 700
tau = 1 / (4 * np.pi ** 2)
radial_order = angular_order = order = 4

data = nib.load(os.path.join(DATA_DIR,"data.nii.gz")).get_data()

bvals, bvecs = read_bvals_bvecs(os.path.join(DATA_DIR,"bvals"), os.path.join(DATA_DIR,"bvecs"))
gtab = gradient_table(bvals, bvecs)

dti_mask = nib.load(os.path.join(DATA_DIR,"mask.nii.gz")).get_data()
dti_fa = nib.load(os.path.join(DATA_DIR,"dti_FA.nii.gz")).get_data()
gm_mask = nib.load(os.path.join(DATA_DIR,"fast_pve_1.nii.gz")).get_data()
mask = np.logical_and(dti_mask, np.logical_and(gm_mask > 0.95, dti_fa < 0.2)).astype('int')

In [7]:
def accumulate_shore(shore_coeff, mask):
    N = shore_coeff.shape[0]
    shore_accum = np.zeros(mic_shore.get_size(radial_order, angular_order))
    accum_count = 0
    nan_count = 0
    for i in range(N):
        if mask[i] == 0:
            continue
        for s in shore_coeff[i,:]:
            if np.isnan(s):
                nan += 1
                break
        else:
            shore_accum += shore_coeff[i,:]
            accum_count += 1

    print(accum_count, "voxel")
    if nan_count > 0:
        print(nan_count, "nans")
    if accum_count == 0:
        return shore_accum
    return shore_accum / accum_count

In [33]:
NX, NY, NZ = data.shape[0:3]
N = NX*NY*NZ
mic_complete = mic_shore.fit(data.reshape((N,-1)), radial_order, angular_order, gtab, zeta, tau, mask.flatten())

  0.0%


/home/olivier/Devel/venvs/bonndit/lib/python3.5/site-packages/dipy/core/geometry.py:129: RuntimeWarning: invalid value encountered in true_divide
  theta = np.arccos(z / r)
/home/olivier/Devel/mic-tools/packages/dwmri/shore.py:154: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  r = la.lstsq(M, data[i, :])



 28.8%

 38.1%

 47.7%

 55.8%

 65.3%

 73.4%

 84.7%

   done   


In [34]:
mic_gm_response = accumulate_shore(mic_complete, mask.flatten())

32432 voxel


In [64]:
from bonndit.michi import fields, dwmri
dti_fa, meta = fields.load_scalar(os.path.join(DATA_DIR, "dti_FA.nii.gz"))
dti_mask, _ = fields.load_scalar(os.path.join(DATA_DIR, "mask.nii.gz"))
gm_mask, _ = fields.load_scalar(os.path.join(DATA_DIR, "fast_pve_1.nii.gz"))
data, gtabm, meta = dwmri.load(os.path.join(DATA_DIR, "data.nii.gz"))
#gtab = gradient_table(gtabm.bvals, gtabm.bvecs)

In [65]:
sm = bd.ShoreModel(gtab, order, zeta, tau)
bonndit_complete = sm._get_response(data, mask)

In [66]:
bonndit_gm_response = sm._accumulate_shore(bonndit_complete, mask)

In [67]:
((bonndit_complete.flatten() - mic_complete.flatten())==0).all()

True

In [70]:
(bonndit_gm_response - mic_gm_response)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

In [17]:
mic_complete_reshaped = mic_complete.reshape((N,-1))
mic_gm_response_reshaped = accumulate_shore(mic_complete_reshaped, mask.flatten())

32432 voxel


In [18]:
bonndit_gm_response - mic_gm_response_reshaped

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

In [29]:
mic_gm_response

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])